<a href="https://colab.research.google.com/github/miroorc/Capstone-Project/blob/main/Ecthr_B_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade datasets
!pip install -U accelerate
!pip install transformers
!pip install evaluate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import datasets
from datasets import load_dataset,load_metric
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification,TrainingArguments, Trainer, EvalPrediction
from datasets import load_metric
from sklearn.metrics import accuracy_score, f1_score, recall_score
import torch

In [ ]:
#downlaod dataset
ecthrb = load_dataset("lex_glue", "ecthr_b")

#split into train, test, validation
ecthrb['train']=ecthrb['train'].select(range(1000))
ecthrb['test']=ecthrb['test'].select(range(100))
ecthrb['validation']=ecthrb['validation'].select(range(100))

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

In [ ]:
train=ecthrb['train']
validation=ecthrb['validation']
test=ecthrb['test']

In [ ]:
num_classes = 10
#function to convert labels into one-hot-encodigns
def one_hot_encode_multi(labels, num_classes):
    encoded_labels = np.zeros((len(labels),num_classes), dtype=int)
    for i, sublist in enumerate(labels):
        encoded_labels[i, sublist] = 1
    return encoded_labels

In [ ]:
#call function above and convert trian, test, validation data to one-hot-encoded vectors
train_one_hot=one_hot_encode_multi(train['labels'],10)
train_one_hot = train_one_hot.tolist()
train=train.add_column(name="labels_one_hot", column=train_one_hot)


test_one_hot=one_hot_encode_multi(test['labels'],10)
test_one_hot = test_one_hot.tolist()
test=test.add_column(name="labels_one_hot", column=test_one_hot)


validation_one_hot=one_hot_encode_multi(validation['labels'],10)
validation_one_hot = validation_one_hot.tolist()
validation=validation.add_column(name="labels_one_hot", column=validation_one_hot)

In [ ]:
train_one_hot

In [ ]:
#function that reformats the data in order to feed into the model
def format_text(text):
    text_format = []
    text = [[re.sub(r"^\d+\.\s*", "", s) for s in sublst] for sublst in text]
    for each in text:
        text_format.append(' '.join(each))
    prefix = 'Judge the following case: '
    text_format = [prefix + s for s in text_format]
    return text_format

In [ ]:
len(train["text"][0])

In [ ]:
#call train, test, validation data to the function above.
train_text_format=format_text(train['text'])
train=train.remove_columns('text')
train=train.add_column(name="text", column=train_text_format)
ecthrb['train']=train


test_text_format=format_text(test['text'])
test=test.remove_columns('text')
test=test.add_column(name="text", column=test_text_format)
ecthrb['test']=test


validation_text_format=format_text(validation['text'])
validation=validation.remove_columns('text')
validation=validation.add_column(name="text", column=validation_text_format)
ecthrb['validation']=validation

In [ ]:
ecthrb

DatasetDict({
    train: Dataset({
        features: ['labels', 'labels_one_hot', 'text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['labels', 'labels_one_hot', 'text'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['labels', 'labels_one_hot', 'text'],
        num_rows: 100
    })
})

In [ ]:
ecthrb=ecthrb.remove_columns('labels')
ecthrb=ecthrb.rename_column('labels_one_hot', 'labels')

In [ ]:
ecthrb["train"]

Dataset({
    features: ['labels', 'text'],
    num_rows: 1000
})

In [ ]:
#define model and tokenizer
model_ckpt = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")

In [ ]:
#function to convert text to tokens
def tokenize_and_encode(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
cols = ecthrb["train"].column_names
cols.remove("labels")
#maps all data to convert to tokens
ds_enc = ecthrb.map(tokenize_and_encode, batched=True, remove_columns=cols)

In [ ]:
ds_enc.set_format("torch")

In [ ]:
num_labels=10
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification")
model.to('cuda')

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, recall_score
from transformers import EvalPrediction
import torch
import numpy as np

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    # next, use threshold to turn them into integer predictions
    sigmoid = torch.nn.Sigmoid() #call sigmoid
    probs = torch.nn.functional.softmax(torch.tensor(predictions), dim=-1).numpy()
    probss = sigmoid(torch.Tensor(probs))
    y_pred = np.zeros(probss.shape)
    y_pred[np.where(probss >= threshold)] = 1

    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')

    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'recall': recall}
    return metrics

#caluclate prediction and return the result
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
#define arguments
batch_size = 1
metric_name = "f1"

args = TrainingArguments(
    f"output",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

#define trainer
trainer = Trainer(model=model,
                  args=args,
                  train_dataset=ds_enc["train"],
                  eval_dataset=ds_enc["validation"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.evaluate()

# **Experiment 1**

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, recall_score
from transformers import EvalPrediction
import torch
import numpy as np

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
# first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid() #call sigmoid
    probss = sigmoid(torch.Tensor(predictions))
    probss = np.round(probss)
    y_pred = np.zeros(probss.shape)
    # next, use threshold to turn them into integer predictions
    y_pred[np.where(probss >= threshold)] = 1

    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'recall': recall}
    return metrics

#function to get prediction and its results
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result



batch_size = 1
metric_name = "f1"

args = TrainingArguments(
    f"output",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=ds_enc["train"],
                  eval_dataset=ds_enc["validation"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.evaluate()